# 05: Duality and Arrangements

*Authors: Felix Espey*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup
3. Duality  
    3.1. Setup  
    3.2. Visualisation  
    3.3. Algorithm(s)
4. Arragements
5. References

## 1. Introduction

This chapter contains two connected topics, **Duality** and **Arrangements**.

**Duality** is an alternative approach to think about points and lines, representing one as the other. This allows us to use algorithms, that solve problems on lines, to also solve problems on points and vice versa. For example, if 3 points are on a line, the dual lines of those points all intersect in a single point. So checking if 3 points are on a line is equivalent to checking if 3 lines intersect in a single point

An **Arrangement** $\mathcal{A}$ is a subdivision of the plane induced by a set $L$ of lines. It has vertices (intersection of the lines of $L$), edges (segments of the lines of $L$) and faces (area in-between the lines of $L$).

Many problems on a set of points can be solved by first transforming those points into lines and then building the Arrangement on them.

This Notebook will first give an introduction to Duality to make the concept clear and go over Arrangements afterward. Lastly, we will look at some example applications of Duality that use Arrangements to solve otherwise difficult problems much faster.

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations, as well as our module for visualization purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [1]:
from modules import (
    VisualisationTool, PointSetInstance, LineSetInstance, LineSegmentSetInstance, DualityPointsMode, DualityLineMode, DualityLineSegmentMode
    )
from modules.geometry import Point, Line, LineSegment

## 3. Duality

In the intro, we went over how **Duality** allows us to represent points and lines and vice versa. In the lecture, we went over how this conversion works exactly, but let's repeat it here.

 - A **point** $p = (x, y)$ becomes the line $p^* := p_x x - p_y$. $p*$ is then called the dual of $p$.
You can see that the x-coordinate of the point becomes the slope of the line, while the y-coordinate becomes the offset of the line in the y direction.
 - A **line** $l = mx + b$ becomes the point $l^* = (m, -b)$. The slope becomes the x-coordinate and the negated y-offset
 becomes the y-coordinate.
 - A **Line Segment** defined by two points $p_1$ and $p_2$ becomes the two cones between the two dual lines $p^*_1$ and $p^*_2$ of those points. 

The transformation from one object to the other preserves certain properties:
- Self inverse: When Transforming an object $x$ into its dual $x^*$, the dual $x^{**}$ of $x^*$ is the original object $x$ again.
- Vertical distances are preserved: Given a point $p$ and a line $l$ that are a vertical distance $m$ apart, the duals $p^*$ and $l^*$ are also a vertical distance $m$ apart.
- Relative positions are inverted: Given a point $p$ that is below/on/above a line $l$, the dual line $p^*$ is above/through/below the dual point $l^*$
- Intersections: If two lines $l_1, l_2$ intersect in a point $p$, the dual line $p^*$ of the point will go through the dual points $l^*_1, l^*_2$ of the lines
- Collinear: Given three points $p_1, p_2, p_3$ on a line $l$, the dual lines $p_1^*, p_2^*, p_3^*$ will all intersect in the dual point $l^*$

The concept itself is pretty easy to understand, but since it can be hard to visualize, so we will set up a visual aid.

### 3.1 Visualization

We will now set up our tool to visualize duality.  The transformation of objects into their respective duals is very simple, we just set up a method each for points, lines and line segments. We then use special drawing modes that not only draw the initial input but also the corresponding dual. To make sure the objects and their duals can be easily distinguished from other objects and duals, we will draw each pair in a different color.

In [2]:
def dual_point_(p : Point) -> Line:
    return Line(Point(0,-p.y), Point(1000, 1000 * p.x -p.y))

def dual_line(l : Line) -> Point:
    return Point(l.slope(), -l.y_from_x(0))

def dual_lineSegment_(ls : LineSegment) -> tuple[Line,Line]:
    return [dual_point_(ls.upper), dual_point_(ls.lower)]

pI = PointSetInstance(DualityPointsMode())
pI._default_number_of_random_points = 5
pointToLine = VisualisationTool(400, 400, pI)

lI = LineSetInstance(DualityLineMode())
lI._default_number_of_random_points = 10
lineToPoint = VisualisationTool(400, 400, lI)

lsI = LineSegmentSetInstance(DualityLineSegmentMode())
lsI._default_number_of_random_points = 4
lineSegmentToLine = VisualisationTool(400, 400, lsI)

After setting up the visualization above, we can now actually draw everything. Usually, the canvas has its origin in the bottom left corner, so only positive coordinates are visible. To allow visualization of negative values as well, all points drawn will be offset such that the origin is in the center. The canvas will also show the x- and y-axis, the first point is drawn.

Due to the number of cleary distinguishable colors being limited, only 9 different colors are visible at a time. If more than 9 different colors would be needed the remaining objects are not drawn.

In [3]:
pointToLine.display()
lineToPoint.display()
lineSegmentToLine.display()

## 4. Arrangements

An **Arrangement** $\mathcal{A}(L)$ is a subdivision of the plane induced by a set $L$ of lines. It has vertices (intersection of the lines of $L$), edges (segments of the lines of $L$) and faces (area in-between the lines of $L$).

The outer edges and faces of an Arrangement are unbounded. For edges, this happens because the lines that make the edge have some last intersection with the other lines of $L$. The edge after that intersection has no other endpoint. The outer faces are also unbounded, as they have no additional line limiting their size.

Since DCELs can only store bounded edges and faces, we generate a bounding box $\mathcal{B}$ around $L$ and only consider the parts of lines inside this box. This adds additional vertices where the lines intersect with the bounding box. We also get new edges as the boundary of the box becomes part of the DCEL.

The steps to computing the Arrangement $\mathcal{A}(L)$ are as follows:
1. compute the bounding box $\mathcal{B}(L)$
2. initialize DCEL with the edges, vertices and interior face from the bounding box
3. incrementally add the lines of $L$ into the DCEL

To start, we set up some example problems for later and set up our visualization again.

In [4]:
from modules.geometry import Rectangle, Point, Line, EPSILON, dual_points, dual_lines, Orientation as ORT
from itertools import combinations
from modules.data_structures.animation_objects import BoundingBoxAnimator, DCELAnimator
from modules.visualisation import VisualisationTool, LineSetInstance, BoundingBoxMode, DCELMode, offset_lines, offset_points
from modules.data_structures import Vertex, HalfEdge, Face, DoublyConnectedEdgeList as DCEL

#Visualisation tool
visualisation_tool = VisualisationTool(400, 400, LineSetInstance())
bounding_box_mode = BoundingBoxMode()
dcel_mode = DCELMode()

#Setting up examples
left, right, lower, upper, offset = 50, 350, 50, 350, 50
four_lines = [Line(Point(10,200), Point(200, 390)), Line(Point(200, 390), Point(390,200)), Line(Point(390,200), Point(200, 10)), Line(Point(10,200), Point(200, 10))]
diagonals_1 = [Line(Point(48, 200), Point(238, 390)), Line(Point(86, 200), Point(276, 390)),
               Line(Point(124,200), Point(314, 390)), Line(Point(162,200), Point(352, 390)),
               Line(Point(200,200), Point(390, 390)), Line(Point(200,162), Point(390, 352)),
               Line(Point(200,124), Point(390, 314)), Line(Point(200,86), Point(390, 276)),
               Line(Point(200,48), Point(390, 238))]
diagonals_2 = [Line(Point(200, 390), Point(390, 200)), Line(Point(200, 352), Point(390, 162)),

               Line(Point(200, 314), Point(390,124)), Line(Point(200, 276), Point(390,86)),
               Line(Point(200, 238), Point(390,48)), Line(Point(162, 238), Point(352,48)),
               Line(Point(124, 238), Point(314,48)), Line(Point(86, 238), Point(276,48)),
               Line(Point(48, 238), Point(238,48)), Line(Point(10, 238), Point(200,48))]
line_grid = four_lines + diagonals_1 + diagonals_2
line_star = four_lines + [Line(Point(10, 10), Point(390,390)), Line(Point(10, 390), Point(390,10)), Line(Point(10, 200), Point(390,200))]

#Registering the examples
visualisation_tool.register_example_instance("four_lines", four_lines)
visualisation_tool.register_example_instance("grid_example", line_grid)
visualisation_tool.register_example_instance("star_example", line_star)

### 4.1 Bounding box

As stated above, the first step is to calculate the bounding box $\mathcal{B}$ of the set of lines $L$.

This bounding box should contain all vertices of the Arrangement $\mathcal{A}(L)$ of $L$. Since the vertices of $\mathcal{A}(L)$ are just the intersection points of the lines in $L$, what we actually need to find are the maximum and minimum x and y coordinates of the intersections. Then we can use them as corner points for an axis aligned box that contains all vertices.

To find the intersection points, we could use a sweep line algorithm, but in this case we will just go with a brute force Algorithm that runs in quadratic time. In theory this is super easy, but in practice we need to be careful with special cases. The steps are as follows:

1. First, we need to find all Intersections between the lines. One could do this using a line sweep, but since it does not change the overall asymptotical runtime, we will use a brute force approach.
2. Search the list of intersections for the biggest and smallest x and y coordinates.
3. Build initial DCEL from the corners of the bounding box

We start by looping over all combinations of lines and find their intersection. The intersection of two lines has 3 different possible types: None if they are parallel but not identical, Line if they are identical and Point otherwise. For our case, only the Point intersections are relevant, so we ignore the other two types. We also add in an additional constraint that the intersections we use must be inside the visible area. This is only so we can guarantee to fully see the bounding box and would normally not be part of the algorithm.

In [5]:


def compute_bounding_box(lines:set[Line], visible_area : Rectangle) -> BoundingBoxAnimator:
    bounding_box_animator : BoundingBoxAnimator = BoundingBoxAnimator()
    for line1, line2 in combinations(lines, 2):
        intersection = line1.intersection(line2)
        if type(intersection) is Point and visible_area.isInside(intersection):
            bounding_box_animator.update(intersection)
    return bounding_box_animator

def compute_bounding_box_alg(lines:set[Line]) -> BoundingBoxAnimator:
    return compute_bounding_box(lines, Rectangle(Point(1, 1), Point(399, 399)))


visualisation_tool.register_algorithm("Bounding Box", compute_bounding_box_alg, bounding_box_mode)
visualisation_tool.display()

### 4.2 Arrangement

First, we need to be able to find intersections between Lines and Edges, which are effectively Line Segments. To avoid duplicate code, we just transform the Edge into a Line by removing the endpoints. If the intersection is a Point, we then have 3 different possibilities:

1. The intersection is very one of the endpoints
2. The intersection is in-between the endpoints
3. The intersection is outside the endpoints

For the first case, we have to use Epsilon inequalities, so any intersection that is close enough to a Vertex will use the Vertex as the intersection point. In the second case, we create a new Vertex that splits the edge. In the third case, continue with the next edge, since we do not have any intersection with the current edge.

In [6]:
def line_edge_intersection(line : Line, edge : HalfEdge) -> Vertex | Point | None:
    e_as_line = Line(edge.origin.point, edge.destination.point)
    intersection = line.intersection(e_as_line)
    if type(intersection) is Point:
        if(intersection.distance(edge.origin.point) < EPSILON):
            return edge.origin
        elif(intersection.distance(edge.destination.point) < EPSILON):
            return edge.destination
        elif(intersection.orientation(edge.origin.point, edge.destination.point) is ORT.BETWEEN):
            return intersection
    return None

Next up are the methods that actually calculate the Arrangement.

We start with *calc_boundingBox_intersection*. This method takes in the DCEL and a Line and finds the intersection of the Line and the Bounding Box of the DCEL. Since we changed the way the bounding box is calculated and ignore intersections outside the visible are, there could be lines that completely miss the Bounding Box. This means we can have three possible situations.

- The Line intersects the bounding box in a Vertex
- The Line intersects the bounding box in an edge
- The Line does not intersect the bounding box

In the third case we return None and continue with the next Line, and in the first case we can just return the Vertex. In the second case, we need to return both the edge and the intersection Point, since we need the Point to know where to add a new Vertex and the Edge to continue following the line through the DCEL. 

The method then starts at the bottom left Vertex, which gets stored when the DCEL gets set up initially. Next we find the edge from that vertex that is adjacent to the outer face and follow the *next* pointers of the edges to go around the outer boundary of the DCEL. At every edge, the intersection between the edge and the Line is calculated and returned if any is found. Once the initial Vertex is reached again, the algorithm can return None, as the Line did not intersect any of the edges of the bounding box.

In [7]:
def calc_boundingBox_intersection(line : Line, dcel : DCELAnimator) -> Vertex | tuple[HalfEdge, Point] | None:
    outer_face = dcel.get_outer_face()
    v = dcel.get_bottom_left()
    #find first edge of bounding box
    edge = v.edge
    while (not (edge.incident_face is outer_face)):
        edge = edge.twin.next
    #check edges on bounding box for intersection
    while(not (edge.destination is v)):
        intersection = line_edge_intersection(line, edge)
        if (not intersection is None):
            if (type(intersection) is Vertex):
                return intersection
            else:
                return (edge, intersection)
        edge = edge.next
    return None

Once we have found a line-edge intersection, we need to follow the line through the arrangement. From the previous step we either have a Vertex or an Edge and a Point on that Edge. If the line intersected and Edge there are no additional steps needed, we move to the twin of the edge and have found the face that gets split by the Line. But if the Line happens to go through an already existing Vertex, we now need to find which of the faces adjacent to the vertex is the one we will need to split.

To find the correct face, we will use the two methods below. The *point_between_edge_and_next* method checks if a point is in the cone spanned by an edge and the next edge in the cycle. If the point is in the given cone, this also means that any line from the vertex to that point must also split the face between the edges. In the *find_leaving_edge* method, we use this method to check every ingoing edge of the Vertex. Since the Line goes through the Vertex, we know it splits exactly two faces around it, one of which we came from and the other being the next face we have to split. Since we know the Face we were at previously, we just check if we found a different Face and only return the corresponding Edge in that case.

The choice of Points given to the methods are also important. We need to make sure that the Points are from different sides of the Vertex, otherwise we can miss the correct Face. To make sure we always have Points on different sides of the Vertex, we move the two points that make up the Line to outside the visible area on different sides. This way it is guaranteed that one point will be left or above the vertex while the other is right or below.

In [8]:
def point_between_edge_and_next(point: Point, edge: HalfEdge) -> bool:
        edge_0, edge_1 = edge, edge.next
        edge_0_origin, edge_0_destination = edge_0.origin.point, edge_0.destination.point
        edge_1_origin, edge_1_destination = edge_1.origin.point, edge_1.destination.point
        
        if edge_0.twin is edge_1:
            return True
        #point is left of both edges
        caseA1 = point.orientation(edge_0_origin, edge_0_destination) == ORT.LEFT and point.orientation(edge_1_origin, edge_1_destination) == ORT.LEFT# Case A
        #point is left of the first edge and edges make a right turn
        caseA2 = point.orientation(edge_0_origin, edge_0_destination) == ORT.LEFT and edge_1_destination.orientation(edge_0_origin, edge_0_destination) == ORT.RIGHT
        #point is left of second edge and edges make a right turn
        caseB = (point.orientation(edge_1_origin, edge_1_destination) == ORT.LEFT and edge_0_origin.orientation(edge_1_origin, edge_1_destination) == ORT.RIGHT)
        return caseA1 or caseA2 or caseB # Case C (where?)

def find_leaving_edge(v : Vertex, f : Face, line : Line) -> HalfEdge | None:
    e_start = v.edge.twin
    e_cur = e_start
    if(point_between_edge_and_next(line.p1, e_cur) and not (f is e_start.incident_face)):
        return e_cur
    if(point_between_edge_and_next(line.p2, e_cur) and not (f is e_start.incident_face)):
        return e_cur
    e_cur = e_cur.next.twin
    while(not(e_cur is e_start)):
        if(point_between_edge_and_next(line.p1, e_cur) and not (f is e_cur.incident_face)):
            return e_cur
        if(point_between_edge_and_next(line.p2, e_cur) and not (f is e_cur.incident_face)):
            return e_cur
        e_cur = e_cur.next.twin
    return None

The last step to calculate the Arrangement is to actually trace the Line.

We get a starting edge either from the intersection between the Line and the bounding box or by calculating it with the methods above. In the first case, we then have to move to the twin of the edge to enter the next face. Here we follow the *next* pointers of the edge until an edge intersects with the Line. Here we again have to distinguish between intersections in vertices and edges. If the intersection is a Vertex, we add an Edge to the previous intersection and then find the next edge by using the methods above again. If the intersection is on the edge, we instead split it by adding a new Vertex and then add an edge to the previous vertex. The algorithm then continues with the twin of the edge. This process is then repeated until we reach the outer face again.

In [9]:
def trace_line(e_init : HalfEdge, v_init : Vertex, line : Line, dcel : DCELAnimator):
    if e_init is None:
        e_init = find_leaving_edge(v_init, dcel.get_outer_face(), line)
        if e_init is None:
            return
        e_cur = e_init
    else:
        e_cur = e_init.twin
    v_cur = v_init
    while(not e_cur.incident_face is dcel.get_outer_face()):
        intersection = None
        while(intersection is None or e_cur.origin is v_cur or e_cur.destination is v_cur):
            e_cur = e_cur.next
            dcel.animate_edge(e_cur.origin.point, e_cur.destination.point)
            intersection = line_edge_intersection(line, e_cur)
        if type(intersection) is Vertex:
            dcel.add_edge(v_cur.point, intersection.point)
            v_cur = intersection
            e_cur = find_leaving_edge(intersection, e_cur.incident_face, line)
            if e_cur is None:
                return
        elif type(intersection) is Point:
            v_new = dcel.add_vertex_on_edge(intersection, e_cur)
            dcel.add_edge(v_new.point, v_cur.point)
            v_cur = v_new
            e_cur = e_cur.twin

def computeDCEL(lines : set[Line]) -> DCELAnimator:
    bbA : BoundingBoxAnimator = compute_bounding_box_alg(lines)
    dcel : DCELAnimator = DCELAnimator(bbA.bounding_box)
    if dcel.illformed:
        return dcel
    for line in lines:
        dcel.highlight_line(line)
        intersection = calc_boundingBox_intersection(line, dcel)
        if type(intersection) is Vertex:
            trace_line(None, intersection, line, dcel)
        elif type(intersection) is tuple:
            v = dcel.add_vertex_on_edge(intersection[1], intersection[0])
            trace_line(intersection[0], v, line, dcel)
        dcel.unhighlight_line(line)
    return dcel

visualisation_tool.register_algorithm("Arrangement", computeDCEL, dcel_mode)
visualisation_tool.display()


# 5. Examples

In the next section, we will go over two examples that use duality to find more efficient solutions to problems than a classical approach. The examples are *3 Points on a Line* and *Min Area Triangle*, which are taken from the lecture but were not explored in detail there.

## 5.1 Three Points on a Line

Our first example is an algorithm that takes a set of points and checks if any three points within that set are on a line. First, let's remember that since our points use float coordinates, they will never be exactly on a line, so we will have to use epsilon inequalities. A simple algorithm would calculate a line between every pair of points and see if any of the other points is on that line, but this would have a worst-case runtime of $O(N^3)$.

With duality, we can solve the problem much faster. The dual formulation of this problem is: Are there three lines that intersect in the same point. To calculate this, we will calculate the Arrangement of the dual lines and stop once the same Vertex is intersected by three lines. Since an Arrangement only has complexity of $O(n^2)$, this is much faster than the classical approach.

To know when to stop, let's see how adding a new line impacts the number of edges. For vertices that are no on the bounding box, every line going through that vertex will generate two edges, so if a Vertex has more than four edges, there are at least three Lines intersecting in that Vertex. Vertices on the bounding box start with two edges from the bounding box, but every Line going through those vertices only generates one edge. This means that, as with inner vertices, any outer vertex that has four or more edges is intersected by three lines. 

The code below is very similar to the Arrangement from before, but has added checks to see if newly created edges lead to the vertex having five or more edges total. When such a vertex is found, the algorithm terminates and the vertex as well as the points corresponding to the intersecting lines are highlighted.

In [10]:
def computeDCEL_3Points(lines : set[Line], return_points : bool = False) -> tuple[DCELAnimator, Point, list[Point]]:
    bbA : BoundingBoxAnimator = compute_bounding_box_alg(lines)
    dcel : DCELAnimator = DCELAnimator(bbA.bounding_box)
    if dcel.illformed:
        return (dcel, None)
    for line in lines:
        dcel.highlight_line(line)
        intersection = calc_boundingBox_intersection(line, dcel)
        if type(intersection) is Vertex:
            #intersection with bounding box is a vertex
            if len(intersection.outgoing_edges()) >= 4:
                #the vertex already has two other lines going through it
                pointsOnLine = offset_points(dual_lines(offset_lines([Line(e.origin.point, e.destination.point) for e in intersection.outgoing_edges()], False)), True)
                pointsOnLine = pointsOnLine + offset_points(dual_lines(offset_lines([line], False)), True)
                return (dcel, intersection.point, pointsOnLine)
            vert = trace_3_points_on_line(None, intersection, line, dcel)
            if not (vert is None):
                #during line trace, a three-line intersection was found
                pointsOnLine = offset_points(dual_lines(offset_lines([Line(e.origin.point, e.destination.point) for e in vert.outgoing_edges()], False)), True)
                return (dcel, vert.point, pointsOnLine)
        elif type(intersection) is tuple:
            #interesection with bounding box is on an edge
            v = dcel.add_vertex_on_edge(intersection[1], intersection[0])
            vert = trace_3_points_on_line(intersection[0], v, line, dcel)
            if not (vert is None):
                #during line trace, a three-line intersection was found
                pointsOnLine = offset_points(dual_lines(offset_lines([Line(e.origin.point, e.destination.point) for e in vert.outgoing_edges()], False)), True)
                return (dcel, vert.point, pointsOnLine)
        dcel.unhighlight_line(line)
    return (dcel, None, None)

def trace_3_points_on_line(e_init : HalfEdge, v_init : Vertex, line : Line, dcel : DCELAnimator) -> Vertex | None:
    if e_init is None:
        e_init = find_leaving_edge(v_init, dcel.get_outer_face(), line)
        e_cur = e_init
    else:
        e_cur = e_init.twin
    v_cur = v_init
    while(not e_cur.incident_face is dcel.get_outer_face()):
        intersection = None
        while(intersection is None or e_cur.origin is v_cur or e_cur.destination is v_cur):
            e_cur = e_cur.next
            dcel.animate_edge(e_cur.origin.point, e_cur.destination.point)
            intersection = line_edge_intersection(line, e_cur)
        if type(intersection) is Vertex:
            dcel.add_edge(v_cur.point, intersection.point)
             #check if after adding the new edge, the vertex now has 5 or more outgoing edges
            if len(intersection.outgoing_edges()) >= 5:
                return intersection
            v_cur = intersection
            e_cur = find_leaving_edge(intersection, e_cur.incident_face, line)
            if e_cur is None:
                return None
        elif type(intersection) is Point:
            v_new = dcel.add_vertex_on_edge(intersection, e_cur)
            dcel.add_edge(v_new.point, v_cur.point)
            v_cur = v_new
            e_cur = e_cur.twin
    return None

def three_points_on_line(points : set[Point]) -> DCELAnimator:
    points = offset_points(points, False)
    duals : set[Line] = dual_points(points)
    duals = offset_lines(duals, True)
    dcel, point, points_on_line = computeDCEL_3Points(duals)
    if point is not None:
        dcel.add_point(point)
    if points_on_line is not None:
        for p in points_on_line:
            dcel.add_point(p)
    return dcel
    
example_1 = [Point(181, 51), Point(221, 50), Point(180, 351), Point(220, 350)]
example_2 = example_1 + [Point(180, 220), Point(210, 230), Point(240, 240)]
instance = PointSetInstance(DualityPointsMode())
instance._default_number_of_random_points = 5
dcel_mode = DCELMode(point_radius=3)
visualisation_tool_2 = VisualisationTool(400, 400, instance)
visualisation_tool_2.register_algorithm("3 Points on Line", three_points_on_line, dcel_mode)
visualisation_tool_2.register_example_instance("3_points_not_on_line", example_1)
visualisation_tool_2.register_example_instance("3_points_on_line", example_2)

The visualization below displays the *3 points on a line* algorithm. It is important to note that it is very hard to actually draw three points on a line by hand, so the example instances are an important tool here.

In [11]:
visualisation_tool_2.display()

## Minimal area triangle

The next example is called min-area triangle. The problem is to find 3 points in a given set that together form the triangle with the smallest possible area. A naive algorithm would have to check all possible 3 points combinations, but using duality and arrangements we get a faster solution.

We start by constructing the DCEL and stop if we happen to find three points on a line, as they would form a triangle of zero area. For this we can reuse the code from the previous example. If we do not find three points on a line, we go through every face and check all edge/vertex combinations of the face. This generates candidates for the smallest triangle, for which we calculate the area and return the smallest.

In [12]:
from modules.data_structures.animation_objects.dcel_animator import MinAreaTriangleAnimator
from modules.visualisation import SmallestAreaTriangleMode


def get_candidates(faces : list[Face], animator : MinAreaTriangleAnimator) -> list[list[Point]]:
    candidates = []
    for face in faces:
        for edge in face.outer_half_edges():
            animator.add_edge(edge.origin.point, edge.destination.point)
            for vertex in face.outer_vertices():
                animator.animate_point(vertex.point)
                if (edge.origin.x < vertex.x and edge.destination.x > vertex.x) or (edge.origin.x > vertex.x and edge.destination.x < vertex.x):
                    candidates.append(
                        offset_points(
                            dual_lines(
                                offset_lines(
                                    [Line(e.origin, e.destination) for e in ([edge] + [edge for edge in vertex.incident_edges() if edge.incident_face is face])]
                                , False)
                            ), 
                        True)
                    )
            animator.remove_edge()
    return candidates

def get_smallest_candiate_index(candidates : list[list[Point]], animator : MinAreaTriangleAnimator) -> int:
    areas = []
    min = 10000000
    index = -1
    for candidate in candidates:
        p1 : Point = candidate[0]
        p2 : Point = candidate[1]
        p3 : Point = candidate[2]
        animator.animate_triangle(p1,p2,p3)
        area = 1/2 * abs(p1.x * (p2.y - p3.y) + p2.x * (p3.y - p1.y) + p3.x * (p1.y - p2.y))
        areas.append(area)
        if area < min:
            min = area
            index = len(areas)-1
    return index


def min_area_triangle(points : set[Point]) -> MinAreaTriangleAnimator:
    points = offset_points(points, False)
    duals : set[Line] = dual_points(points)
    duals = offset_lines(duals, True)
    dcel, point, points = computeDCEL_3Points(duals)
    animator = MinAreaTriangleAnimator(dcel)
    if points is not None:
        animator.smallest_triangle = points
        return animator
    else:
        candidates = get_candidates(dcel.faces(), animator)
        index = get_smallest_candiate_index(candidates, animator)
        if index != -1:
            animator.smallest_triangle = candidates[index]
    return animator
    
visualisation_tool_2.register_algorithm("smallest triangle", min_area_triangle, SmallestAreaTriangleMode())
visualisation_tool_2.display()

# 5.References

\[1\] Mark de Berg, Otfried Cheong, Marc van Kreveld, and Mark Overmars. *Computational Geometry: Algorithms and Applications*, 3rd edition. 2008.